# Advanced Housing Price Prediction

In this project we will try to perform various data cleaning processing and build a simple Linear Regression model to predict housing prices on AmesHousing dataset.\
The source of the data can be found in this location [HERE](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627)

##### Import the required libraries

In [1]:
# Data Manipulation and Visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Modelling and Testing
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [2]:
# Some housekeeping - allowing large columns
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100000)
pd.set_option('max_colwidth', 1000)

##### Read input dataset

In [3]:
inp_data = pd.read_csv('./data/AmesHousing.tsv', delimiter='\t')
inp_data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


### Data Pre-processing

Now, we will start cleaning up the dataset.\
Basically, we will check for the following things:
<ol>
    <li>If percentage of null records exceed 5% -> Remove the feature</li>
    <li>For text columns drop rows where there are missing values</li>
    <li>If nulls are less than 5%, we will fill them with most appropriate values</li>
    <li>Convert the text columns into categorical features</li>
    <li>If the features are not of much use, drop them</li>
    <li>Try creating new features if possible</li>
</ol>

#### 1. Null Values

In [4]:
# count the number of null values
null_val = inp_data.isnull().sum()

In [5]:
# find out columns where null values are greater than 5%
null_gt_5 = null_val[(null_val > len(inp_data)/20)].sort_values()
null_gt_5

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

So we can see that there are 11 features which have more than 5% null values. We shall drop these features and move forward

In [6]:
semi_clean = inp_data.drop(null_gt_5.index, axis=1)


#### 2. Drop null rows only for text columns

In [7]:
# find the text columns which have null values
text_null = semi_clean.select_dtypes(include=['object']).isnull().sum()
text_null_gt0 = text_null[(text_null > 0)].sort_values()
text_null_gt0

Electrical         1
Mas Vnr Type      23
Bsmt Qual         80
Bsmt Cond         80
BsmtFin Type 1    80
BsmtFin Type 2    81
Bsmt Exposure     83
dtype: int64

In [8]:
# drop these columns with null values
semi_clean = semi_clean.drop(text_null_gt0.index, axis=1)
semi_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 64 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Area         2930 non-null   int64  
 5   Street           2930 non-null   object 
 6   Lot Shape        2930 non-null   object 
 7   Land Contour     2930 non-null   object 
 8   Utilities        2930 non-null   object 
 9   Lot Config       2930 non-null   object 
 10  Land Slope       2930 non-null   object 
 11  Neighborhood     2930 non-null   object 
 12  Condition 1      2930 non-null   object 
 13  Condition 2      2930 non-null   object 
 14  Bldg Type        2930 non-null   object 
 15  House Style      2930 non-null   object 
 16  Overall Qual     2930 non-null   int64  
 17  Overall Cond  

Now our dataset is mostly null removed. Doing so, we started  with 83 features and now we have reduced it to 64 features only.\
However, there are a few features with Null values still.\
We will fill this with the most common value

#### 3. Fill missing values with most common value

In [9]:
null_val_intfloat = semi_clean.select_dtypes(include=['int64','float']).isnull().sum()
null_val_intfloat_gt0 = null_val_intfloat[(null_val_intfloat > 0)].sort_values()
null_val_intfloat_gt0

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

We will fill the data into these 9 features, however, we need to know what values to fill.\
It is very difficult to say which value might work for which features.\
In order to do so, we need to take a deeper dive into the dataset information.

From the dataset description we could find the following:
<ul>
    <li>BsmtFin SF 1: Finished Square Feet </li>
    <li>BsmtFin SF 2: Type 2 Finished Square Feet </li>
    <li>Bsmt Unf SF: Unfinished Basement Square Feet </li>
    <li>Total Bsmt SF: Total SF of basement area </li>
    <li>Garage Cars: Size of garage in car capacity </li>
    <li>Garage Area: Size of garage in square feet </li>
    <li>Bsmt Full Bath: Basement full bathrooms </li>
    <li>Bsmt Half Bath: Basement half bathrooms </li>
    <li>Mas Vnr Area: Masonry veneer area </li>
</ul>

It is interesting to note that most of these values are continuous or in some ranges.

When dealing with replacing missing values we have two choices:
<ol>
    <li>If the feature is in some continuous range of values, we can allow to replace the missing values with the average (mean) of the distribution </li>
    <li>If the feature is some categorical feature, we should pick the most dominant category (mode) of the distribution</li>
</ol>

In this case, we will simply replace all the missing values with the mean, since these are continuous variables.

In [10]:
# Replace missing values with mean
semi_clean = semi_clean.fillna(semi_clean.mean())

In [11]:
# check if any missing values are still left
rem_nulls = semi_clean.isnull().sum()
print(rem_nulls[rem_nulls > 0].sort_values())

Series([], dtype: int64)


In [12]:
clean_data = semi_clean.copy()

So we have cleaned up the missing values. Now we will perform Feature Engineering

### Feature Engineering

On taking a deeper-dive into the dataset, we note the following: 

There are a lot of features which needs to be engineered and some of the features can be combines. For instance,

<ul>
    <li>Bsmt Full Bath: Basement full bathrooms </li>
    <li>Bsmt Half Bath: Basement half bathrooms </li>
    <li>Full Bath: Full bathrooms above grade </li>
    <li>Half Bath: Half baths above grade </li>
</ul>
These features can be combined into two single features

<ul>
    <li>Mo Sold: Month Sold (MM) </li>
    <li>Yr Sold: Year Sold (YYYY) </li>
    <li>Sale Type: Type of sale </li>
    <li>Sale Condition: Condition of sale </li>
</ul>
These features can be dropped as there is no requirement for what type of sale has been processed

<ul>
    <li>Year Built: Original construction date </li>
    <li>Year Remod/Add: Remodel date (same as construction date if no remodeling or additions) </li>
</ul>    
These features needs to be engineered to a more relevant feature
    

In [13]:
# combining features to get age at selling time
age_of_house_sold = clean_data['Yr Sold'] - clean_data['Year Built']
age_of_house_sold.head()

0    50
1    49
2    52
3    42
4    13
dtype: int64

In [14]:
#  checking for negative data
age_of_house_sold[age_of_house_sold < 0]

2180   -1
dtype: int64

In [15]:
# combining features to get age at selling time
age_of_house_remod = clean_data['Yr Sold'] - clean_data['Year Remod/Add']
#  checking for negative data
age_of_house_remod[age_of_house_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [16]:
# dropping the columns which have been extracted above
clean_data.drop('Year Built', axis=1, inplace=True)
clean_data.drop('Year Remod/Add', axis=1, inplace=True)
clean_data.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC',
       'Central Air', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF',
       'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val',
       'Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition', 'SalePrice'],
  

In [17]:
# adding new features
clean_data['age_of_house_sold'] = age_of_house_sold
clean_data['age_of_house_remod'] = age_of_house_remod
clean_data.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC',
       'Central Air', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF',
       'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val',
       'Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition', 'SalePrice',
   

In [18]:
# dropped rows with negative values
clean_data.drop([1702,2180,2181], axis=0, inplace=True)

In [19]:
# recombining half and full bathroom features
clean_data['basement_bathrooom'] = clean_data['Bsmt Full Bath'] + (clean_data['Bsmt Half Bath'])*0.5
clean_data['bathrooom'] = clean_data['Full Bath'] + (clean_data['Half Bath'])*0.5

In [20]:
# dropping extracted features
clean_data.drop(['Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath'], axis=1, inplace=True)

In [21]:
# dropping sale related columns
clean_data.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'], axis=1, inplace=True)

In [22]:
clean_data.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC',
       'Central Air', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF',
       'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val',
       'SalePrice', 'age_of_house_sold', 'age_of_house_remod',
       'basement_bathrooom', 'bathrooom'],
      dtype='object')

In [23]:
# dropping some other features which are not that important
clean_data.drop(['PID', 'Order'], axis=1, inplace=True)

### Linear Regression Model and RMS Error

In [26]:
# def a function for training/testing
def train_and_test(df):
    # splitting dataframe into train/test segments
    train = df[0:1460]
    test = df[1460:]
    
    # selecting only numeric values from train/test sets
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    features = numeric_train.columns.drop('SalePrice')
    linreg = linear_model.LinearRegression()
    linreg.fit(train[features], train['SalePrice'])
    predictions = linreg.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [27]:
train_and_test(clean_data)

70888.75727097962

### Feature Selection

Now, as our dataset contains a large number of features, we will try to find suitable features to make our data more efficient and terse. \
First, we will try to find the numerical (integer and float) features and derive the correlation among them.

In [28]:
# numerical features
numr_feat = clean_data.select_dtypes(include=['int64', 'float'])

# correlation among these features
num_data_corr = numr_feat.corr()['SalePrice'].abs().sort_values(ascending=False)
num_data_corr

SalePrice             1.000000
Overall Qual          0.801206
Gr Liv Area           0.717596
Garage Cars           0.648411
Total Bsmt SF         0.643601
Garage Area           0.641675
1st Flr SF            0.635185
bathrooom             0.581357
age_of_house_sold     0.558979
age_of_house_remod    0.534985
Mas Vnr Area          0.510611
TotRms AbvGrd         0.498574
Fireplaces            0.474831
BsmtFin SF 1          0.438928
Wood Deck SF          0.328183
Open Porch SF         0.316262
basement_bathrooom    0.270144
2nd Flr SF            0.269601
Lot Area              0.267520
Bsmt Unf SF           0.182248
Bedroom AbvGr         0.143916
Enclosed Porch        0.128685
Kitchen AbvGr         0.119760
Screen Porch          0.112280
Overall Cond          0.101540
MS SubClass           0.085128
Pool Area             0.068438
Low Qual Fin SF       0.037629
3Ssn Porch            0.032268
Misc Val              0.019273
BsmtFin SF 2          0.006000
Name: SalePrice, dtype: float64

In [ ]:
# we will keep only those columns which have correlation higher than 0.3
clean_data.drop(num_data_corr[num_data_corr < 0.4].index, axis=1, inplace=True)

#### Categorical Features

In order to deal with the categorical feature selection we will perform the following operations
<ol>
    <li>Create a list of categorical column names </li>
    <li>Convert numerical features that should be categorical </li>
    <li>Drop categorical columns with more than 10 unique values </li>
    <li>Drop categorical columns with unique values under 2 and an very skewed distribution </li>
    <li>Create dummies for remaining categorical columns </li>
</ol>

Following are the categorical variables:
<ul>
<li>Street</li> <li>Lot Shape</li> <li>Land Contour</li> <li>Utilities</li> <li>Lot Config</li> <li>Land Slope</li> <li>Neighborhood</li> <li>Condition 1</li> <li>Condition 2</li> <li>Bldg Type</li> <li>House Style</li> <li>Roof Style</li> <li>Roof Matl</li> <li>Exterior 1st</li> <li>Exterior 2nd</li> <li>Foundation</li> <li>Heating</li> <li>Central Air</li> <li>Functional</li> <li>Paved Drive</li> 
</ul>

Followinng are the numerical variables which needs to be converted into categorical variables:
<ul> <li>MS SubClass</li> <li>Exter Qual</li> <li>Exter Cond</li> <li>Heating QC</li> <li>Kitchen Qual</li> </ul>

In [30]:
# Creating list of categorical features
cat_features = ['Street', 'Lot Shape', 'Land Contour', 'Utilities',
                'Lot Config', 'Land Slope', 'Neighborhood',
                'Condition 1', 'Condition 2', 'Bldg Type',
                'House Style', 'Roof Style','Roof Matl',
                'Exterior 1st','Exterior 2nd','Foundation',
                'Heating', 'Central Air', 'Functional', 'Paved Drive',
                'MS SubClass', 'Exter Qual', 'Exter Cond', 'Heating QC',
                'Kitchen Qual'
               ]

In [31]:
# Creating list of cat features left in dataset
transform_cat_features = []
for col in cat_features:
    if col in clean_data.columns:
        transform_cat_features.append(col)

In [32]:
# Number of unique values in each cat column
unique_cat_vals = clean_data[transform_cat_features].apply(lambda col: len(col.value_counts())).sort_values()
unique_cat_vals

Street           2
Central Air      2
Utilities        3
Land Slope       3
Paved Drive      3
Lot Shape        4
Land Contour     4
Exter Qual       4
Exter Cond       5
Heating QC       5
Kitchen Qual     5
Bldg Type        5
Lot Config       5
Roof Style       6
Foundation       6
Heating          6
Condition 2      8
Functional       8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
MS SubClass     16
Exterior 2nd    17
Neighborhood    28
dtype: int64

In [33]:
# Removing columns with more than 10 unique values
nonuniq_cols = unique_cat_vals[unique_cat_vals > 10].index
clean_data.drop(nonuniq_cols, axis=1, inplace=True)

In [34]:
# Listing remaining cat columns for skewed data
remaining_cat_cols = []
for col in cat_features:
    if col in clean_data.columns:
        remaining_cat_cols.append(col)
        
# Exploring remaining cat columns with less than 5 unique values
cat_val_counts = clean_data[remaining_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
skewed_cols = cat_val_counts[cat_val_counts < 5].index
skewed_cols

Index(['Street', 'Central Air', 'Utilities', 'Land Slope', 'Paved Drive',
       'Lot Shape', 'Land Contour', 'Exter Qual'],
      dtype='object')

In [36]:
for col in skewed_cols:
    print(clean_data[col].value_counts())
    print('\n')

Pave    2915
Grvl      12
Name: Street, dtype: int64


Y    2731
N     196
Name: Central Air, dtype: int64


AllPub    2924
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64


Gtl    2786
Mod     125
Sev      16
Name: Land Slope, dtype: int64


Y    2649
N     216
P      62
Name: Paved Drive, dtype: int64


Reg    1859
IR1     976
IR2      76
IR3      16
Name: Lot Shape, dtype: int64


Lvl    2632
HLS     120
Bnk     115
Low      60
Name: Land Contour, dtype: int64


TA    1799
Gd     988
Ex     105
Fa      35
Name: Exter Qual, dtype: int64




We can note that the features 'Street' and 'Utilities' have highly skewed data. So, we will drop these features and proceed

In [40]:
# dropping skewed columns
clean_data.drop(['Street', 'Utilities'], axis=1, inplace=True)

In [42]:
# converting all cat columns that remain to categorical
# Creating list of cat features left in dataset
transform_cat_features = clean_data.select_dtypes(include=['object'])

for col in transform_cat_features:
    clean_data[col] = clean_data[col].astype('category')

In [43]:
# creating drop list of category features
cat_cols = clean_data.select_dtypes(include=['category'])

# creating df of dummy categories
dummies = pd.get_dummies(clean_data.select_dtypes(include=['category']))

# Create dummies and add back to dataframe
data_final = pd.concat([clean_data, dummies], axis=1)

# Drop original dummy categories
data_final.drop(cat_cols, axis=1, inplace=True)

In [44]:
train_and_test(data_final)

57112.987063581335

## Preparing for KFold and Cross Validation Testing

We will first form all the above steps performed into functions and then perform the KFold Testing

In [46]:
# def a function for cleaning the data
def transform_features(df):
    # creating a df of null counts in each category
    null_counts = df.isnull().sum()
    # identifying columns with more than 5% null values
    drop_na_cols = null_counts[(null_counts > len(df)/20)].sort_values()
    # drop those columns from dataframe
    df.drop(drop_na_cols.index, axis=1, inplace=True)
    
    # identifying only text columns
    missing_text_vals = df.select_dtypes(include=['object']).isnull().sum()
    replace_text_cols = missing_text_vals[(missing_text_vals > 0)].sort_values()
    # dropping columns with missing values in text columns
    df.drop(replace_text_cols.index, axis=1, inplace=True)
    
    # identifying only numerical columns
    missing_vals = df.select_dtypes(include=['int64','float']).isnull().sum()
    replace_cols = missing_vals[(missing_vals > 0)].sort_values()
    ## Fill missing values with mean
    df.fillna(df.mean(), inplace=True)
    
    # Engineering Age of house feature
    years_sold = df['Yr Sold'] - df['Year Built']
    # Dropping Year Built column
    df.drop('Year Built', axis=1, inplace=True)
    # Engineering Years Before Remodel column
    df['years_since_remodel'] = df['Yr Sold'] - df['Year Remod/Add']
    # Dropping Year Remod column
    df.drop('Year Remod/Add', axis=1, inplace=True)
    # verifying that new feature contains positive values
    years_since_remod = df['years_since_remodel']
    # Creating new columns
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    # Drop rows with negative values
    df.drop([2180, 1702, 2181], axis=0, inplace=True)
    # Combining bathroom features
    df['Bath'] = df['Full Bath'] + (df['Half Bath']*0.5)
    df['Bsmt Bath'] = df['Bsmt Full Bath'] + (df['Bsmt Half Bath']*0.5)
    # Dropping old bathroom features
    df.drop(['Full Bath','Half Bath','Bsmt Full Bath','Bsmt Half Bath'], axis=1, inplace=True)
    # Dropping columns which are proxy for sale
    df.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'], axis=1, inplace=True)
    # Dropping columns which aren't useful for our model
    df.drop(['PID','Order'], axis=1, inplace=True)
    
    return df

In [50]:
# def a function for feature selection
def select_features(df):
    ## Keeping correlated numerical features
    # Selecting only numerical columns
    df_num = df.select_dtypes(include=['int64','float'])
    # Checking for correlation
    df_corr_coef = df_num.corr()['SalePrice'].abs().sort_values(ascending=False)
    df_corr_coef
    # Keeping only columns with correlation coefficient larger than 0.4
    df.drop(df_corr_coef[df_corr_coef < 0.4].index, axis=1, inplace=True)
    
    # Creating list of categorical features
    cat_features = ['Street', 'Lot Shape', 'Land Contour', 'Utilities',
                    'Lot Config', 'Land Slope', 'Neighborhood',
                    'Condition 1', 'Condition 2', 'Bldg Type',
                    'House Style', 'Roof Style','Roof Matl',
                    'Exterior 1st','Exterior 2nd','Foundation',
                    'Heating', 'Central Air', 'Functional', 'Paved Drive',
                    'MS SubClass', 'Exter Qual', 'Exter Cond', 'Heating QC',
                    'Kitchen Qual'
                   ]
    # Creating list of cat features left in dfset
    transform_cat_features = []
    for col in cat_features:
        if col in df.columns:
            transform_cat_features.append(col)
    
    ## Dropping cat features with too many unique values
    # Number of unique values in each cat column
    unique_cat_vals = df[transform_cat_features].apply(lambda col: len(col.value_counts())).sort_values()
    # Removing columns with more than 10 unique values
    nonuniq_cols = unique_cat_vals[unique_cat_vals > 10].index
    df.drop(nonuniq_cols, axis=1, inplace=True)
    
    ## Dropping cat features with skewed values
    # dropping skewed columns
    df.drop(['Street', 'Utilities'], axis=1, inplace=True)
    
    # converting all cat columns that remain to categorical
    # Creating list of cat features left in dfset
    transform_cat_features = df.select_dtypes(include=['object'])
    # Converting these features to category type
    for col in transform_cat_features:
        df[col] = df[col].astype('category')
    
    ## Get dummies for categorical features and drop them
    # creating drop list of category features
    cat_cols = df.select_dtypes(include=['category'])
    # creating df of dummy categories
    dummies = pd.get_dummies(df.select_dtypes(include=['category']))
    # Create dummies and add back to dfframe
    df_final = pd.concat([df, dummies], axis=1)
    # Drop original dummy categories
    df_final.drop(cat_cols, axis=1, inplace=True)

    return df_final

In [51]:
# def a function for training/testing
def train_and_test(df, k=0):
    # splitting dataframe from target column
    numeric_df = df.select_dtypes(include=['int64', 'float'])
    features = numeric_df.columns.drop('SalePrice')
    lr=linear_model.LinearRegression()
    
    # holdout validation
    if k == 0:
        # splitting dataframe into train/test segments
        train = df[0:1460]
        test = df[1460:]
    
        lr.fit(train[features], train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse = np.sqrt(mse) 
        return rmse
    
    # simple cross validation
    if k == 1:
        # Randomizing all rows from df
        shuffled_df = df.sample(frac=1, )
        # splitting dataframe into train/test segments
        train = df[0:1460]
        test = df[1460:]
        
        # training data
        lr.fit(train[features], train['SalePrice'])
        predictions_one = lr.predict(test[features])
        mse_one = mean_squared_error(test['SalePrice'], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        # test data
        lr.fit(test[features], test['SalePrice'])
        predictions_two = lr.predict(train[features])
        mse_two = mean_squared_error(train['SalePrice'], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        # averaging rmse
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one, rmse_two)
        return avg_rmse
    
    # K-fold validation
    else:
        # building K-folds
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            # creating train/test set for fold
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            # fitting and predicting
            lr.fit(train[features], train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test['SalePrice'], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        # Averaging rmse values
        avg_rmse = np.mean(rmse_values)
        print(avg_rmse)
        return avg_rmse

### Holdout Testing

In [52]:
# Holdout testing
# Reading in original data into new df
data = pd.read_csv('./data/AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)
# Running regression model
train_and_test(data_final, k=0)

36632.403066029794

### Cross Validation Testing

In [53]:
# Simple cross validation
# Reading in original data into new df
data = pd.read_csv('./data/AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)
# Running regression model
train_and_test(data_final, k=1)

36632.403066029794 30931.92440139434


33782.16373371206

### KFold Testing

In [54]:
# Reading in original data into new df
data = pd.read_csv('./data/AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)

# testing to optimize k-fold value
k_range = list(range(2,100,1))
rmses = {}
for i in k_range:
    rmse = train_and_test(data_final, k=i)
    rmses[i] = rmse

33697.362021760404
33345.2299069662
33337.06396490855
33113.757970178325
33225.54686737526
32742.921383141173
32959.079907685154
33043.02562298861
32722.512710083432
32504.943750736817
32584.903190296114
32578.797050686266
32452.875488733636
32514.30787942693
31977.99633557202
32239.372985372553
32632.51142482915
32321.319272418365
32392.940207985277
32324.18209895441
32367.090897557824
32104.53714426326
32279.040234430413
32150.322210290808
31997.07696702292
32055.372052772636
31997.63342873714
31895.206924515132
31992.38329273368
31991.008658255687
32007.894826358446
32019.894604363457
31697.596839183858
31709.67267135146
31736.16823809831
31663.859514849242
31795.26957783205
31595.945036007866
31794.635984122335
31566.821424293877
31661.790235076267
31699.904622042774
31605.07180699106
31671.28848450354
31754.449361066065
31574.35252624288
31627.02231409516
31421.827923440826
31438.37744913229
31675.718685326494
31412.804819949313
31452.78821718223
31344.90050141134
31109.0465639457

In [55]:
# Using min() + list comprehension + values() 
# Finding min value keys in dictionary 
temp = min(rmses.values()) 
res = [key for key in rmses if rmses[key] == temp] 
  
# printing result  
print("Model with minimum rmse is: " + "k = " + str(res) + ", rmse = " + str(temp))

Model with minimum rmse is: k = [98], rmse = 30495.773468817508
